# Create params_with_metrics
```
input: paramsMatrix.csv
output: params_with_metrics.csv
```
Add these columns to params matrix:
- Feature drop rate
- Heuristic quality
- Planning depth
- ELO rating

In [1]:
import numpy as np
import pandas as pd
from fourinarowfunctions import *

%load_ext autoreload
%autoreload 2

Provide the directory where the data is kept

In [2]:
direc = '../data/'

Read parameters and calculate feature drop rate and heuristic quality

In [3]:
df = pd.read_csv(direc + 'paramsMatrix.csv')
df = df.drop("Unnamed: 0", axis = 1)
params = np.array([expand_params(r) for r in df.values[:,-10:].astype(float)])

df['feature drop rate'] = params[:,-1]
df['heuristic quality'] = np.apply_along_axis(get_heuristic_quality,1,params)

We compute planning depth on the cluster. First, export the parameters

In [4]:
np.savetxt(direc + 'params.txt', params)

Create a directory on the cluster and move params.txt into it. Edit the compute_planning_depth.sh shell script so that the directory is correct, and change the number of jobs to the number of lines in params.txt (in this case 370). Run the script, then zip the depth folder with ```zip -r depth.zip depth```, move it back to your local computer and unzip in the directory where paramsMatrix.csv is stored. 

In [5]:
depth = np.vstack([np.loadtxt(direc + f"depth/depth_fourinarow-dev_{i // 5}_{i % 5 + 1}.txt") for i in range(len(params))])
df['planning depth'] = depth

Calculate Elo

In [6]:
data = load_data(direc + "trialdata.csv")

drop test_im:R_2YspUds9g1Pyp6J
drop test_im:R_1LYfkmETpZRD3kn
drop debug:R_3G7Yq6aginiNva2
drop test_im2:R_2YspUds9g1Pyp6J
drop im_full_test:assign_full_test
drop test_im3:R_2YspUds9g1Pyp6J
drop null:R_emsNDLsvWMLPb7X
drop test_im_ff:test_FF_win
drop test_im_edge:test_edge_win
drop twindemo:noas
drop null:R_2qCyb2CeLwcRxH2
drop null:R_25HjvhOMhc79iaa
drop debugYZNI1G:debugDVJFSO
drop test_BO_11:R_1eXnot6X58rnPMv
drop null:R_2zhw77FMGEb36zK
drop debugOCOGDF:debug2TDLTY
drop test_im_ff2:test_FF2_win
drop null:R_2aV9fwk4CR3GjBg
drop debugQWOQ1C:debugOQ8IEY
drop CPTest:R_1QxPuBVlzMbldXa
drop CPTest:R_1FsoM2AkWpLn1Qw
drop im_full_test:ssign_full_test
drop just_a_test_:test
drop debugfull:noas
drop null:R_vou3qrtMy6aHysp
drop debugscreenshot:noas
Loaded 192 participants


In [7]:
def categoryHist(username, data):
    table = np.empty([0,6])
    category = 2
    for event in data:
        if event['event_type'] == 'user move':
            color = event['event_info']['user_color']
        if event['event_type'] == 'adjust level':
            category = event['event_info']['category']
        if event['event_type'] == 'end game':
            result = event['event_info']['result']
            resultCode = 1 if result == 'win' else -1 if result == 'opponent win' else 0
            table = np.vstack((table, [username, event['event_info']['game_num'], int(category), event['event_info']['level'], color, resultCode]))
    return table

In [8]:
# create dataframe
forEloData = np.empty([0,6]) 
for username, userdata in data.items():
    forEloData = np.vstack((forEloData, categoryHist(username, userdata)))
forElo = pd.DataFrame(data = forEloData, columns = ['subject', 'gameNumber', 'category', 'level', 'user_color', 'outcome'])
forElo["outcome"] = forElo["outcome"].astype(int)
print(f"Elo data for {len(pd.unique(forElo['subject']))} subjects")

Elo data for 192 subjects


In [9]:
results = [[row['subject'].split(':')[1],str(row['level']),row['outcome']] if row['user_color']=='black' else
           [str(row['level']),row['subject'].split(':')[1],-row['outcome']] for _,row in forElo.iterrows()]
create_bayeselo_input(results, direc + 'games.pgn')

In [10]:
bayeselo_direc = './'

elo = run_bayeselo(bayeselo_direc,[direc + 'tournament_results_short.pgn', direc + 'games.pgn'])
elo = {key : val for key,val in elo.items() if key.startswith('R_')}
df["elo"] = df["subject"].apply(lambda x: x.split(":")[1]).map(elo)

All the metrics are in the datafrome now. Let's save it.

In [11]:
df.to_csv(direc + 'params_with_metrics.csv', index = False)